## Pyspark Project
This is a practice project which i apply what i learn while learning apache spark (pyspark and sparksql). I assume that i am a data engineer and this is a task to perform tasks such as reading, analyzing, transforming and loading  using pyspark and sparksql.
### Objectives
- Read datasets
- Do some transformation
- Load data in different sources (Hive data warehouse and HDFS (Hadoop Distributed File system))

### Download data

In [54]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv  
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv  
!rm -r data
!mkdir data
!mv dataset1.csv data/dataset1.csv
!mv dataset2.csv data/dataset2.csv

--2025-12-02 19:11:14--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset1.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4115 (4.0K) [text/csv]
Saving to: ‘dataset1.csv’

dataset1.csv        100%[===================>]   4.02K  --.-KB/s    in 0s      

2025-12-02 19:11:15 (1.01 GB/s) - ‘dataset1.csv’ saved [4115/4115]

--2025-12-02 19:11:15--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0225EN-SkillsNetwork/labs/data/dataset2.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.10

In [28]:
import pandas as pd
from pyspark.sql import SparkSession, functions as F
import findspark
from pyspark.sql.types import StructField, StructType, IntegerType, DateType, StringType

In [29]:
findspark.init()

In [30]:
spark = SparkSession.builder.getOrCreate()

### Why the following cell is important
Because the DateType() in pyspark support only the following format `yyyy-MM-dd` without the following the date column after reading the file becoming NULL becuase spark.type DateType cannot read it. So the following cell transform the original csv file to be in the correct format. 

In [31]:
d1 = pd.read_csv('data/dataset1.csv')
d2 = pd.read_csv('data/dataset2.csv')
def transform_date (x):
    l= x.split("/")
    year=l[len(l)-1]
    month= l[1]
    day=l[0]
    return f"{year}-{month}-{day}"

d1['date_column'] = d1['date_column'].apply(transform_date)
d2['transaction_date'] = d2['transaction_date'].apply(transform_date)


d1.to_csv('data/dataset1.csv',index=False)
d2.to_csv('data/dataset2.csv',index=False)

### Defining the schema of the data frame

In [32]:

df1_schema = StructType(
    [
        StructField('customer_id', IntegerType(),True),
        StructField('date_column', DateType(),True),
        StructField('amount', IntegerType(),True),
        StructField('description', StringType(),True),
        StructField('location', StringType(),True),
    ]
)

df2_schema = StructType(
    [
        StructField('customer_id', IntegerType(),True),
        StructField('transaction_date', DateType(),True),
        StructField('value', IntegerType(),True),
        StructField('note', StringType(),True)
    ]
)


df1 = spark.read.csv('data/dataset1.csv',header=True, schema=df1_schema)
df2 = spark.read.csv('data/dataset2.csv',header=True, schema=df2_schema)


In [33]:
print(f"Schema of DataFrame 1")
df1.printSchema()
print(f"Schema of DataFrame 2")
df2.printSchema()

Schema of DataFrame 1
root
 |-- customer_id: integer (nullable = true)
 |-- date_column: date (nullable = true)
 |-- amount: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- location: string (nullable = true)

Schema of DataFrame 2
root
 |-- customer_id: integer (nullable = true)
 |-- transaction_date: date (nullable = true)
 |-- value: integer (nullable = true)
 |-- note: string (nullable = true)



### Add new columns to each dataframe
1. Add `year` column to dataframe 1
2. Add `quarter` column to dataframe 2

In [34]:
df1 = df1.withColumn('Year', F.year(F.col('date_column')))
df2 = df2.withColumn('Quarter', F.quarter(F.col('transaction_date')))

df1.show(2)
df2.show(2)

+-----------+-----------+------+-----------+--------+----+
|customer_id|date_column|amount|description|location|Year|
+-----------+-----------+------+-----------+--------+----+
|          1| 2022-01-01|  5000| Purchase A| Store A|2022|
|          2| 2022-02-15|  1200| Purchase B| Store B|2022|
+-----------+-----------+------+-----------+--------+----+
only showing top 2 rows
+-----------+----------------+-----+------+-------+
|customer_id|transaction_date|value|  note|Quarter|
+-----------+----------------+-----+------+-------+
|          1|      2022-01-01| 1500|Note 1|      1|
|          2|      2022-02-15| 2000|Note 2|      1|
+-----------+----------------+-----+------+-------+
only showing top 2 rows


25/12/02 19:05:36 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: customer_id, transaction_date, value, notes
 Schema: customer_id, transaction_date, value, note
Expected: note but found: notes
CSV file: file:///home/omar/me/Data-Engineering/big-data/Final-Projects/data/dataset2.csv


### Rename Columns in DataFrames
1. In dataframe 1, rename `amount` to be `transaction_amount`
2. In dataframe 2, rename `value` to be `transaction_value`

In [35]:
df1 = df1.withColumnRenamed('amount','transaction_amount')
df2 = df2.withColumnRenamed('value','transaction_value')

print(df1.columns)
print(df2.columns)

['customer_id', 'date_column', 'transaction_amount', 'description', 'location', 'Year']
['customer_id', 'transaction_date', 'transaction_value', 'note', 'Quarter']


### Drop unnecessary Columns
1. In dataframe 1, drop `description` and `location`
2. In dataframe 2, drop `note`

In [36]:
df1 = df1.drop('description','location')
df2 = df2.drop('note')

print(df1.columns)
print(df2.columns)

['customer_id', 'date_column', 'transaction_amount', 'Year']
['customer_id', 'transaction_date', 'transaction_value', 'Quarter']


### Join both dataframes based on `customer_id` with inner join

In [37]:
joined_df = df1.join(df2,on='customer_id', how='inner')

In [38]:
joined_df.columns

['customer_id',
 'date_column',
 'transaction_amount',
 'Year',
 'transaction_date',
 'transaction_value',
 'Quarter']

In [39]:
joined_df.show(5)

+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|Year|transaction_date|transaction_value|Quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|          1| 2022-01-01|              5000|2022|      2022-01-01|             1500|      1|
|          2| 2022-02-15|              1200|2022|      2022-02-15|             2000|      1|
|          3| 2022-03-20|               800|2022|      2022-03-20|             1000|      1|
|          4| 2022-04-10|              3000|2022|      2022-04-10|             2500|      2|
|          5| 2022-05-05|              6000|2022|      2022-05-05|             1800|      2|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
only showing top 5 rows


### Do some filteration tasks on the joined dataframe
- filter records which are have `transaction_amount` greater than 1000 and store the result in new dataframe `filtered-df`

In [40]:
filtered_df = joined_df.filter(joined_df['transaction_amount'] > 1000)
filtered_df.orderBy('transaction_amount',ascending=True).show(5)

+-----------+-----------+------------------+----+----------------+-----------------+-------+
|customer_id|date_column|transaction_amount|Year|transaction_date|transaction_value|Quarter|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
|         22| 2023-10-30|              1200|2023|      2023-10-30|              700|      4|
|         34| 2024-10-30|              1200|2024|      2024-10-30|              700|      4|
|          2| 2022-02-15|              1200|2022|      2022-02-15|             2000|      1|
|         70| 2027-10-30|              1200|2027|      2027-10-30|              700|      4|
|         46| 2025-10-30|              1200|2025|      2025-10-30|              700|      4|
+-----------+-----------+------------------+----+----------------+-----------------+-------+
only showing top 5 rows


In [41]:
#Data is filitered correctly
filtered_df.groupBy('transaction_amount').agg(
    F.count('customer_id').alias('count')
).orderBy('transaction_amount',ascending=False).show()

+------------------+-----+
|transaction_amount|count|
+------------------+-----+
|              6000|    1|
|              5500|    7|
|              5000|    1|
|              4800|    1|
|              4500|    1|
|              4200|    8|
|              3500|    1|
|              3200|    7|
|              3000|    1|
|              2600|    8|
|              2400|    7|
|              2200|    1|
|              1800|    8|
|              1500|    7|
|              1200|    8|
+------------------+-----+



### Aggregate the dataframe by customers and calculate the total `transaction_amount` of each customer who his `transaction_amount` is greater than 1000

In [42]:
total_amount_per_customer = filtered_df.groupBy('customer_id').agg(
    F.sum('transaction_amount').alias('Total_Transaction_Amount')
).orderBy('customer_id')

In [43]:
total_amount_per_customer.show(5)

+-----------+------------------------+
|customer_id|Total_Transaction_Amount|
+-----------+------------------------+
|          1|                    5000|
|          2|                    1200|
|          4|                    3000|
|          5|                    6000|
|          6|                    4500|
+-----------+------------------------+
only showing top 5 rows


### Writing data
- `total_amount_per_customer` dataframe in a Hive table named `customer_totals`
- `filtered_df` in HDFS in parquet format called `filtered_data`

**Why parquet??**

It's a columnar storage file format that is efficient for big data processing because it stores data by columns instead of rows.

In [44]:
#Overwrite the table if exists
total_amount_per_customer.write.mode('overwrite').saveAsTable('customer_totals')

In [45]:
filtered_df.write.mode('overwrite').parquet('filtered-data.parquet')

### Add new columns based on specific conditions
- Add column `high_value` in df1 store **YES** if `transaction_amount` is greater than 5000 otherwise store **NO**

In [46]:
df1 = df1.withColumn('high_value', F.when((df1['transaction_amount'] > 5000),'YES').otherwise('NO'))

In [47]:
df1.show(5)

+-----------+-----------+------------------+----+----------+
|customer_id|date_column|transaction_amount|Year|high_value|
+-----------+-----------+------------------+----+----------+
|          1| 2022-01-01|              5000|2022|        NO|
|          2| 2022-02-15|              1200|2022|        NO|
|          3| 2022-03-20|               800|2022|        NO|
|          4| 2022-04-10|              3000|2022|        NO|
|          5| 2022-05-05|              6000|2022|       YES|
+-----------+-----------+------------------+----+----------+
only showing top 5 rows


### Calculate the average transaction value per quarter

In [48]:
average_value_per_quarter = df2.groupBy('Quarter').agg(
    F.round(F.avg('transaction_value'),2).alias('Average_Transactional_Value')
    ).orderBy('Quarter')

### Write the result `average_value_per_quarter` in Hive table called `quarterly_averages`

In [49]:
average_value_per_quarter.write.mode('overwrite').saveAsTable('quarterly_averages')

###  Calculate the total transaction value per year
- In first dataframe, compute the total transaction value per year and store it in data frame called `total_transaction_val`

In [50]:
df1.columns

['customer_id', 'date_column', 'transaction_amount', 'Year', 'high_value']

In [51]:
total_transaction_val = df1.groupBy('Year').agg(
    F.sum('transaction_amount').alias('Total_Transaction_Amount')
)

### Save the result in a csv file and store it in HDFS under the name of `total_value_per_year`

In [52]:
total_transaction_val.write.mode('overwrite').csv('total_value_per_year.csv')

***Stay Tuned.***

*Wish Me LUCK :)*